## Libraries and Setup

In [1]:
import os
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory

from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
llm_model = "gpt-3.5-turbo"

In [4]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)

## Conversation Buffer Memory

This memory allows for storing messages and then extracts the messages in a variable.

In [6]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [8]:
conversation.predict(input="Hi, my name is Saad Tariq")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: Hi, my name is Saad Tariq
AI:

> Finished chain.


"Hello Saad Tariq! It's a pleasure to meet you. How can I assist you today?"

In [9]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: Hi, my name is Saad Tariq
AI: Hello Saad Tariq! It's a pleasure to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [10]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: Hi, my name is Saad Tariq
AI: Hello Saad Tariq! It's a pleasure to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Saad Tariq. How can I assist you further, Saad Tariq?'

In [11]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: Hi, my name is Saad Tariq
AI: Hello Saad Tariq! It's a pleasure to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Saad Tariq. How can I assist you further, Saad Tariq?


In [12]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: Hi, my name is Saad Tariq\nAI: Hello Saad Tariq! It's a pleasure to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Saad Tariq. How can I assist you further, Saad Tariq?"}

In [13]:
memory = ConversationBufferMemory()

In [14]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [15]:
print(memory.buffer)

Human: Hi
AI: What's up


In [16]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [17]:
memory.save_context(
    {"input": "Not much, just hanging"}, 
    {"output": "Cool"}
)

In [18]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## Conversation Buffer Window Memory

ConversationBufferWindowMemory keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [20]:
memory = ConversationBufferWindowMemory(k=1)

In [21]:
memory.save_context(
    {"input": "Hi"},
    {"output": "What's up"}
)

memory.save_context(
    {"input": "Not much, just hanging"},
    {"output": "Cool"}
)

In [22]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [23]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [24]:
conversation.predict(input="Hi, my name is Saad")

"Hello Saad! It's nice to meet you. How can I assist you today?"

In [25]:
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [26]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to ask?"

## Conversation Token Buffer Memory

ConversationTokenBufferMemory keeps a buffer of recent interactions in memory, and uses token length rather than number of interactions to determine when to flush interactions.

In [28]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

In [29]:
memory.save_context(
    {"input": "AI is what?!"},
    {"output": "Amazing!"}
)
memory.save_context(
    {"input": "Backpropagation is what?"},
    {"output": "Beautiful!"}
)
memory.save_context(
    {"input": "Chatbots are what?"},
    {"output": "Charming!"}
)

In [30]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## Conversation Summary Buffer Memory

ConversationSummaryBufferMemory combines the two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. It uses token length rather than number of interactions to determine when to flush interactions.

In [31]:
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

In [33]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)

In [34]:
memory.save_context(
    {"input": "Hello"},
    {"output": "What's up"}
)
memory.save_context(
    {"input": "Not much, just hanging"},
    {"output": "Cool"}
)
memory.save_context(
    {"input": "What is on the schedule today?"}, 
    {"output": f"{schedule}"}
)

In [35]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to show the latest LLM demo during the lunch meeting.'}

In [36]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [37]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to show the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the LLM demo, it would be great to showcase the latest language model capabilities, such as natural language understanding, text generation, and sentiment analysis. You could also highlight any recent improvements or updates to the model's performance. Additionally, demonstrating how the model can be integrated into different applications or platforms would be beneficial for the customer to see. Let me know if you need any specific examples or data to include in the demo!"

In [38]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to show the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show.\nAI: For the LLM demo, it would be great to showcase the latest language model capabilities, such as natural language understanding, text generation, and sentiment analysis. You could also highlight any recent improvements or updates to the model's performance. Additionally, demonstrating how the model can be integrated into different applications or platforms would be beneficial for the customer to see. Let me know if you need any specific examples or data to include in the demo!"}